# Imports

In [1]:
# Import before anyone else

# import comet_ml
from comet_ml import Experiment

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import shap
import os
import math
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from collections import Counter

# Load Data

In [2]:
df = pd.read_csv('final_df.csv', index_col = None)
print(df.columns)

Index(['game_id', 'season', 'season_type', 'event_id', 'home_team',
       'away_team', 'home_team_side_1st_period', 'attacking_team',
       'attacking_player', 'goalie',
       ...
       'attacking_team_Philadelphia Flyers',
       'attacking_team_Pittsburgh Penguins', 'attacking_team_San Jose Sharks',
       'attacking_team_St. Louis Blues', 'attacking_team_Tampa Bay Lightning',
       'attacking_team_Toronto Maple Leafs',
       'attacking_team_Vancouver Canucks',
       'attacking_team_Vegas Golden Knights',
       'attacking_team_Washington Capitals', 'attacking_team_Winnipeg Jets'],
      dtype='object', length=235)


In [3]:
old_feat = ['game_id', 'season', 'season_type', 'event_id', 'home_team',
       'away_team', 'home_team_side_1st_period', 'attacking_team',
       'attacking_player', 'goalie', 'period', 'period_time', 'goal_ind',
       'shot_ind', 'x_coordinates', 'y_coordinates', 'shot_type', 'empty_net',
       'strength', 'gwg', 'previous_event_x_coordinates',
       'previous_event_y_coordinates', 'previous_event_period_time',
       'previous_event_type', 'previous_event_period', 'distance_from_net',
       'attacking_team_side', 'angle_from_net']

df_feat = df.iloc[:,:60].columns.tolist()

# features extracted in question 4
new_feat = [feat for feat in df_feat if feat not in old_feat]
print(len(old_feat),len(df_feat),len(new_feat) )

28 60 32


# Data Split

In [4]:
df_train = df[df['season'] != 20192020]
df_test = df[df['season'] == 20192020]
df_train_XG = df_train.copy()
df_train_XG2 = df_train.copy()


## 5.1 Train-test

In [5]:
X_1 = df_train[['angle_from_net', 'distance_from_net']]
y_1 = df_train['goal_ind']
#y = y.astype(int)
# (#BLOG)
X_train_1, X_valid_1, y_train_1, y_valid_1 = train_test_split(X_1,y_1,test_size=0.15,random_state=10, stratify=y_1, shuffle = True)

## 5.2 Train-test

In [6]:
X_2 = df_train[new_feat]
y_2 = df_train['goal_ind']
#y = y.astype(int)
X_train_2, X_valid_2, y_train_2, y_valid_2 = train_test_split(X_2,y_2,test_size=0.15,random_state=10, stratify=y_2, shuffle = True)

In [7]:
print(X_train_1.shape)
print(X_train_2.shape)

(282813, 2)
(282813, 32)


In [8]:
df_train["goal_ind"].value_counts()

0    301638
1     31084
Name: goal_ind, dtype: int64

# cometml

In [9]:
experiment = Experiment(
    api_key=os.environ.get('COMET_API_KEY'),
    project_name="xgboost",
    workspace="kleitoun",
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/kleitoun/xgboost/ac7d512d6bcf457383fb932059978551



# XGBoost Classifier

## 5.1

## Experiment url: https://www.comet.ml/kleitoun/milestone-2/6244c288bec04ab28ba6c2c030708813?experiment-tab=metrics

In [10]:
model_1 = XGBClassifier()
 
# fit the model with the training data
model_1.fit(X_train_1,y_train_1)

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
COMET WARNING: You must install graphviz to plot tree


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [11]:
preds_1 = model_1.predict(X_valid_1)

In [12]:
X_train_1.isnull().sum()
np.isinf(X_train_1).values.sum()

0

In [13]:
explainer = shap.Explainer(model_1)
shap_values = explainer(X_train_1)

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


In [ ]:
shap.summary_plot(shap_values, X_train_1)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/mahdikleit/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-078e3d2a6eb1>", line 1, in <module>
    shap.summary_plot(shap_values, X_train_1)
  File "/Users/mahdikleit/opt/anaconda3/lib/python3.8/site-packages/comet_ml/monkey_patching.py", line 318, in wrapper
    new_return_value = callback(
  File "/Users/mahdikleit/opt/anaconda3/lib/python3.8/site-packages/comet_ml/loggers/shap_logger.py", line 54, in after
    experiment._log_figure(
  File "/Users/mahdikleit/opt/anaconda3/lib/python3.8/site-packages/comet_ml/experiment.py", line 1702, in _log_figure
    upload_message = processor.process()
  File "/Users/mahdikleit/opt/anaconda3/lib/python3.8/site-packages/comet_ml/file_uploader.py", line 292, in process
    return self.process_upload_to_be_converted(user_input)
  File "/Users/mahdikleit/opt/anaconda3/l

TypeError: object of type 'NoneType' has no len()

### Evaluation - XGBoost Baseline (5.1)

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, roc_curve, auc
from sklearn.metrics import roc_curve, auc


In [ ]:
y_pred_proba_1 = model_1.predict_proba(X_valid_1)[:,1]
roc_auc =  roc_auc_score(y_valid_1,y_pred_proba_1)
y_pred_1 = model_1.predict(X_valid_1)
accuracy =  accuracy_score(y_valid_1, y_pred_1)
report = classification_report(y_valid_1, y_pred_1, output_dict=True)


In [ ]:
print(roc_auc, accuracy)
report

In [ ]:
metrics = {
    "roc_auc": roc_auc,
    "accuracy": accuracy,
    "classification report": report
}

In [ ]:
experiment.log_metrics(metrics)

In [ ]:
params = {"base_score":0.5, "booster":"gbtree", "colsample_bylevel":1,
              "colsample_bynode":1, "colsample_bytree":1, "enable_categorical":False,
              "gamma":0, "gpu_id":1, "importance_type":None,
              "learning_rate":0.300000012,
              "max_delta_step":0, "max_depth":6, "min_child_weight":1, "missing":np.nan,
              "monotone_constraints":'()', "n_estimators":100, "n_jobs":2,
              "num_parallel_tree":1, "predictor":'auto', "random_state":0,
              "reg_alpha":0, "reg_lambda":1, "scale_pos_weight":1, "subsample":1,
              "tree_method":'exact', "validate_parameters":1, "verbosity":None}

In [ ]:
experiment.log_parameters(params)

In [ ]:
def infer_model(X,y):
    global y_valid 
    global y_pred
    global X_valid
    global predicted_prob
    
    X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.15,random_state=10, stratify=y_1, shuffle = True)

    model_2 = XGBClassifier()
    model_2.fit(X_train,y_train)
    
    y_pred = model_2.predict(X_valid)
    predicted_prob = model_2.predict_proba(X_valid)

#    return y_valid, y_pred, X_valid.shape, predicted_prob


In [ ]:
# Functions for 4 plots

def roc_auc_plot(df_roc,df_auc):
    plt.figure(figsize=(12.5,7.5))
    lw = 3
    color_list = ['darkorange','green','navy','red']
    for i in range(len(list_model_iter)):
        
        plt.plot(df_roc.FPR[i], df_roc.TPR[i],color=color_list[i],lw=lw,label=f'{df_auc.Feature[i]} (area = {df_auc.AUC[i]})')

    plt.plot([0, 1], [0, 1], color="black", lw=lw, label="Ideal Random Baseline",linestyle="--")
    plt.legend(fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.title("ROC CURVE", fontsize=18)
    plt.savefig('ROC_XGBOOST.png')
    plt.show()
    experiment.log_figure(figure=plt,overwrite=False)

def goal_rate_plot(df_perc_prop,n_bins):
    
    fig = plt.figure(figsize=(12.5,7.5))
    color_list = ['darkorange','green','navy','red']
    
    for i in range(len(list_model_iter)):

        ax = sns.lineplot(x = df_perc_prop.pctile[i*n_bins:n_bins*(i+1)-1], y = df_perc_prop.goal_rate[i*n_bins:n_bins*(i+1)-1], label=f'{df_auc.Feature[i]}', color=color_list[i], legend = False, linewidth = 3)

    ax.set_xlim(left=105, right=-5)
    ax.set_ylim(bottom=0, top=.25)
    fig.legend(loc="upper right")
    plt.title(f"Goal Rate v.s. Shot Probability Model Percentile")
    fig.legend(loc="upper right")
    #plt.autoscale()
    fig.show()
    plt.savefig('Goal_Rate_XGBOOST.png')
    experiment.log_figure(figure=fig,overwrite=False)

def cum_rate_plot(df_perc_prop_cum,n_bins):
    
    fig = plt.figure(figsize=(12.5,7.5))
    color_list = ['darkorange','green','navy','red']
    
    for i in range(len(list_model_iter)):

        ax = sns.lineplot(x = df_perc_prop_cum.pctile[i*n_bins:n_bins*(i+1)-1], y = df_perc_prop_cum.cum_goal_rate[i*n_bins:n_bins*(i+1)-1], label=f'{df_auc.Feature[i]}', color=color_list[i], legend = False, linewidth = 3)
 
    ax.set_xlim(left=105, right=-5)
    ax.set_ylim(bottom=0, top=df_perc_prop_cum['cum_goal_rate'].max()+0.05)
    fig.legend(loc="upper right")
    plt.title(f"Cumulative Goal Rate v.s. Shot Probability Model Percentile")
    plt.grid(color='r', linestyle='-', linewidth=0.25)
    fig.legend(loc="upper right")
    #plt.autoscale()
    fig.show()
    plt.savefig('Cum_Goal_XGBOOST.png')
    experiment.log_figure(figure=fig,overwrite=False)

def calibration_plot(df_calib,n_bins,length):
    fig = plt.figure(figsize=(8, 8))
    ax1 = plt.subplot2grid((1, 1), (0, 0), rowspan=1)
    ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
    for i in range(len(list_model_iter)):
        prob_true, prob_pred = calibration_curve(df_calib.goal_ind[int(i*length):int(length*(i+1)-1)], df_calib.Goal_Prob[int(i*length):int(length*(i+1)-1)], n_bins=n_bins)
        ax1.plot(prob_pred, prob_true, "s-",label=f'{df_auc.Feature[i]}')
    ax1.set_ylabel("Fraction of positives")
    ax1.set_ylim([-0.05, 1.05])
    ax1.legend(loc="lower right")
    ax1.set_title('Calibration plots  (reliability curve)')
    ax1.set_xlabel("Mean predicted value")
    ax1.set_ylabel("Fraction of Positives")
    ax1.legend(loc="upper center", ncol=2)
    plt.tight_layout()
    plt.savefig('Calibration_Curve_XGBOOST.png')
    experiment.log_figure(figure=fig,overwrite=False)

In [ ]:
df_train_XG['angle_from_net_abs']=df_train_XG['angle_from_net'].abs()
#df_train2 = df_train.copy()
fpr,tpr,roc_auc = dict(),dict(),dict()
fpr_list,tpr_list,roc_auc_list,feature_list=[],[],[],[]
df_cum_prop1,df_cum_prop2,df_cum_prop3 = [],[],[]

df_perc,goal_count,shot_count,goal_rate,cum_goal_rate,pctile,pctile_prop = [],[],[],[],[],[],[]
goal_count2,pctile2,cum_goal_rate2 = [],[],[]
n_bins, quant = 20,5
list_model_iter = [['distance_from_net'],
                   ['angle_from_net_abs'],
                   ['distance_from_net','angle_from_net_abs'],
                  ]
y = df_train_XG['goal_ind']
predicted_prob_goal,temp_y_valid = [],[]

for ele in list_model_iter:
    feature_list.append(ele)
    if df_train_XG[ele].shape[1] == 1:
        X = df_train_XG[ele].values.reshape(-1,1)
        infer_model(X,y)
        temp_y_valid += y_valid.tolist()
        for prob in predicted_prob:
            predicted_prob_goal.append(prob[1])
    else:
        X = df_train_XG[ele].values
        infer_model(X,y)
        temp_y_valid  += y_valid.tolist()
        for prob in predicted_prob:
            predicted_prob_goal.append(prob[1])
#--------------------Determine ROC curve----------------------------------
length = len(temp_y_valid)/len(list_model_iter)
df_prob = pd.DataFrame(list(zip(temp_y_valid,predicted_prob_goal)), columns = ['goal_ind','Goal_Prob'])
df_prob['shot_count'] = 1
# df_calib is copy of df_prob
df_calib = df_prob.copy()
for m in range(len(list_model_iter)):
    df_prob_f = df_prob.loc[m*length:length*(m+1)-1,:]
    fpr, tpr, _ = roc_curve(df_prob_f['goal_ind'], df_prob_f['Goal_Prob'])
    roc_auc = auc(fpr, tpr)
    fpr_list.append(fpr)
    tpr_list.append(tpr)
    roc_auc_list.append(roc_auc)
# df_prob_f1 is copy of df_prob_f
    df_prob_f1 = df_prob_f.copy()
    df_prob_f1['percentile'] = df_prob_f1['Goal_Prob'].rank(pct=True)
    quantile_list = np.linspace(0,1,n_bins*5+1).round(4).tolist()
    q = df_prob_f1.quantile(quantile_list)
    col = 'Goal_Prob'
    for i in np.arange(quant,(quant*n_bins)+1,quant):
        df_perc = df_prob_f1[((df_prob_f1[col]>=q[col][(i-quant)/100]) & (df_prob_f1[col]<q[col][i/100]))]
        goal_count.append(df_perc['goal_ind'].sum())
        shot_count.append(df_perc['shot_count'].sum())
        goal_rate.append(df_perc['goal_ind'].sum()/df_perc['shot_count'].sum())
        pctile.append(i)
    df_perc_prop = pd.DataFrame(list(zip(goal_count,shot_count,goal_rate,pctile)),columns=['goal_count',"sum_shot_count",'goal_rate','pctile'])
# df_prob_f2 is copy of df_prob_f

    df_prob_f2 = df_prob_f.copy()
    df_prob_f2['percentile'] = df_prob_f2['Goal_Prob'].rank(pct=True)
    q2 = df_prob_f2.quantile(quantile_list)
    total = df_prob_f2['goal_ind'].sum()

    temp,j=0,100
    
    for j in np.arange((quant*n_bins),0,-quant):
        df_perc2 = df_prob_f2[((df_prob_f2[col]>q[col][(j-quant)/100]) & (df_prob_f2[col]<=q[col][j/100]))]
        goal_count2.append(df_perc2.goal_ind.sum())
        temp+=df_perc2.goal_ind.sum()
        cum_goal_rate2.append(temp/total)
        pctile2.append(j)
       
    df_perc_prop_cum = pd.DataFrame(list(zip(goal_count2,cum_goal_rate2,pctile2)),columns=['goal_count','cum_goal_rate','pctile'])
#print(df_perc_prop_cum[:20])


In [ ]:
# Call for functions to plot curves
#----------------------------------------------------------------------------------------------- 
df_roc=pd.DataFrame(list(zip(fpr_list,tpr_list)),columns = ['FPR','TPR'])
df_auc=pd.DataFrame(list(zip(feature_list,roc_auc_list)),columns = ['Feature','AUC'])
roc_auc_plot(df_roc,df_auc)


In [ ]:
import seaborn as sns
#--------------------Determine Goal Rate (#goals / (#no_goals + #goals)) ----------------------------------
goal_rate_plot(df_perc_prop,n_bins)



In [ ]:
#-----------------------------------------------------------------------------------------------
cum_rate_plot(df_perc_prop_cum,n_bins)

In [ ]:
from sklearn.calibration import calibration_curve, CalibrationDisplay
#-----------------------------------------------------------------------------------------------
calibration_plot(df_calib,n_bins,length)

In [ ]:
# experiment.end

## 5.2

In [ ]:
X_train_2.isnull().sum()

In [ ]:
# X_train_2.isinf().sum()
for f in ["distance_from_last_event", "speed", "change_in_angle"]:
    print(np.isinf(X_train_2[[f]]).values.sum())
np.isinf(y_train_2).values.sum()

In [ ]:
X_train_2.replace([np.inf, -np.inf], np.nan, inplace=True)
for f in ["distance_from_last_event", "speed", "change_in_angle"]:
    print(np.isinf(X_train_2[[f]]).values.sum())

In [ ]:
print(X_train_2.isnull().sum())

In [ ]:
X_train_2.fillna(0, inplace=True)
print(X_train_2.isnull().sum())

In [ ]:
model_2 = XGBClassifier()
 
# fit the model with the training data
model_2.fit(X_train_2,y_train_2)

In [ ]:
preds_2 = model_2.predict(X_valid_2)

In [ ]:
preds_2

In [ ]:
def infer_model3(X_train_ele,y_train,X_valid_ele):
    global y_pred
    global predicted_prob
    

    model_3 = XGBClassifier()
    model_3.fit(X_train_ele,y_train)
    
    y_pred = model_3.predict(X_valid_ele)
    predicted_prob = model_3.predict_proba(X_valid_ele)

In [ ]:
df_train_XG.columns


### Performance Plots

In [ ]:
#df_train2 = df_train.copy()
fpr,tpr,roc_auc = dict(),dict(),dict()
fpr_list,tpr_list,roc_auc_list,feature_list=[],[],[],[]
df_cum_prop1,df_cum_prop2,df_cum_prop3 = [],[],[]

df_perc,goal_count,shot_count,goal_rate,cum_goal_rate,pctile,pctile_prop = [],[],[],[],[],[],[]
goal_count2,pctile2,cum_goal_rate2 = [],[],[]
n_bins, quant = 20,5
list_model_iter = [['distance_from_last_event'],
                   ['speed'],
                   ['change_in_angle'],
                  ]
X = df_train_XG2[['distance_from_last_event','speed','change_in_angle']]
y = df_train_XG2['goal_ind']

X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.15,random_state=10, stratify=y_1, shuffle = True)
# print(X_train.columns,X_train.shape)
# print(X_valid.columns,X_valid.shape)
# break
predicted_prob_goal,temp_y_valid = [],[]

for ele in list_model_iter:
    feature_list.append(ele)
    if X_train[ele].shape[1] == 1:
        X_train_ele = X_train[ele].values.reshape(-1,1)
        X_valid_ele = X_valid[ele].values.reshape(-1,1)
        infer_model3(X_train_ele,y_train,X_valid_ele)
        temp_y_valid += y_valid.tolist()
        for prob in predicted_prob:
            predicted_prob_goal.append(prob[1])
    else:
        X_train_ele = X_train[ele].values
        infer_model3(X_train_ele,y_train,X_valid_ele)
        temp_y_valid  += y_valid.tolist()
        for prob in predicted_prob:
            predicted_prob_goal.append(prob[1])
#--------------------Determine ROC curve----------------------------------
length = len(temp_y_valid)/len(list_model_iter)
df_prob = pd.DataFrame(list(zip(temp_y_valid,predicted_prob_goal)), columns = ['goal_ind','Goal_Prob'])
df_prob['shot_count'] = 1
# df_calib is copy of df_prob
df_calib = df_prob.copy()
for m in range(len(list_model_iter)):
    df_prob_f = df_prob.loc[m*length:length*(m+1)-1,:]
    fpr, tpr, _ = roc_curve(df_prob_f['goal_ind'], df_prob_f['Goal_Prob'])
    roc_auc = auc(fpr, tpr)
    fpr_list.append(fpr)
    tpr_list.append(tpr)
    roc_auc_list.append(roc_auc)
# df_prob_f1 is copy of df_prob_f
    df_prob_f1 = df_prob_f.copy()
    df_prob_f1['percentile'] = df_prob_f1['Goal_Prob'].rank(pct=True)
    quantile_list = np.linspace(0,1,n_bins*5+1).round(4).tolist()
    q = df_prob_f1.quantile(quantile_list)
    col = 'Goal_Prob'
    for i in np.arange(quant,(quant*n_bins)+1,quant):
        df_perc = df_prob_f1[((df_prob_f1[col]>=q[col][(i-quant)/100]) & (df_prob_f1[col]<q[col][i/100]))]
        goal_count.append(df_perc['goal_ind'].sum())
        shot_count.append(df_perc['shot_count'].sum())
        goal_rate.append(df_perc['goal_ind'].sum()/df_perc['shot_count'].sum())
        pctile.append(i)
    df_perc_prop = pd.DataFrame(list(zip(goal_count,shot_count,goal_rate,pctile)),columns=['goal_count',"sum_shot_count",'goal_rate','pctile'])
# df_prob_f2 is copy of df_prob_f

    df_prob_f2 = df_prob_f.copy()
    df_prob_f2['percentile'] = df_prob_f2['Goal_Prob'].rank(pct=True)
    q2 = df_prob_f2.quantile(quantile_list)
    total = df_prob_f2['goal_ind'].sum()

    temp,j=0,100
    
    for j in np.arange((quant*n_bins),0,-quant):
        df_perc2 = df_prob_f2[((df_prob_f2[col]>q[col][(j-quant)/100]) & (df_prob_f2[col]<=q[col][j/100]))]
        goal_count2.append(df_perc2.goal_ind.sum())
        temp+=df_perc2.goal_ind.sum()
        cum_goal_rate2.append(temp/total)
        pctile2.append(j)
       
    df_perc_prop_cum = pd.DataFrame(list(zip(goal_count2,cum_goal_rate2,pctile2)),columns=['goal_count','cum_goal_rate','pctile'])
#print(df_perc_prop_cum[:20])


In [ ]:
df_train.angle_from_net.describe()

# Hyper-paramter Tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold

In [ ]:
# A parameter grid for XGBoost
params = {
    'n_estimators': [100, 150],
    'learning_rate': [0.3, 0.2, 0.1],
    'max_depth': [4,6 , 7, 8]
}

In [ ]:
y_train_2.shape

In [ ]:
folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(model_2, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X_train_2,y_train_2), verbose=3, random_state=1001 )

# Here we go
# start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train_2,y_train_2)
# timer(start_time) # timing ends here for "start_time" variable

In [ ]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)

In [ ]:
model_2_hmtuned = XGBClassifier(n_estimators= 150, max_depth= 4, learning_rate= 0.2, subsample= 1.0, min_child_weight= 5, gamma= 5, colsample_bytree= 1.0)
 
# fit the model with the training data
model_2_hmtuned.fit(X_train_2,y_train_2)

In [ ]:
y_pred_proba_2 = model_2.predict_proba(X_valid_2)[:,1]
roc_auc =  roc_auc_score(y_valid_2,y_pred_proba_2)
y_pred_2 = model_2.predict(X_valid_2)
accuracy =  accuracy_score(y_valid_2, y_pred_2)
report = classification_report(y_valid_2, y_pred_2, output_dict=True)



In [ ]:
print(roc_auc, accuracy)

### Evaluation - XGBoost post Hyperparameter Tuning

In [ ]:
y_pred_proba_2 = model_2_hmtuned.predict_proba(X_valid_2)[:,1]
roc_auc =  roc_auc_score(y_valid_2,y_pred_proba_2)
y_pred_2 = model_2_hmtuned.predict(X_valid_2)
accuracy =  accuracy_score(y_valid_2, y_pred_2)
report = classification_report(y_valid_2, y_pred_2, output_dict=True)




In [ ]:
print(roc_auc, accuracy)

In [ ]:
# count examples in each class
counter = Counter(y_pred_2)
print(counter[0] , counter[1])

# To avoid heavy class imbalance problem, tuning class weights scale

In [ ]:
# NOTE: this approach does NOT yield any good results (#BLOG)

weights = [1, 10, 25, 50, 75, 99, 100, 1000]
param_grid = dict(scale_pos_weight=weights)

In [ ]:
# NOTE: this approach does NOT yield any good results


# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# define grid search
# grid = GridSearchCV(estimator=model_2_hmtuned, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# # execute the grid search
# grid_result = grid.fit(X_train_2,y_train_2)

grid_result = RandomizedSearchCV(model_2, param_distributions=param_grid, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X_train_2,y_train_2), verbose=3, random_state=1001 )

grid_result.fit(X_train_2,y_train_2)

# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# NOTE: this approach does NOT yield any good results


scale_pos_weight = 9
model_2_cw = XGBClassifier(n_estimators= 150, scale_pos_weight = scale_pos_weight, max_depth= 4, learning_rate= 0.2, subsample= 1.0, min_child_weight= 5, gamma= 5, colsample_bytree= 1.0)
 
# fit the model with the training data
model_2_cw.fit(X_train_2,y_train_2)

In [ ]:
# NOTE: this approach does NOT yield any good results


y_pred_proba_cw = model_2_cw.predict_proba(X_valid_2)[:,1]
roc_auc =  roc_auc_score(y_valid_2,y_pred_proba_cw)
y_pred_cw = model_2_cw.predict(X_valid_2)
accuracy =  accuracy_score(y_valid_2, y_pred_cw)
report = classification_report(y_valid_2, y_pred_cw, output_dict=True)


In [ ]:
print(roc_auc, accuracy)
report

In [ ]:
from collections import Counter

# count examples in each class
counter = Counter(y_pred_cw)
print(counter[0] , counter[1])
# estimate scale_pos_weight value
# estimate = counter[0] / counter[1]
# print('Estimate: %.3f' % estimate)

### Class weights

In [ ]:
from sklearn.utils import class_weight

# assigining class weight to balanced does NOT yield any good results/makes NO difference (#BLOG)

classes_weights = class_weight.compute_sample_weight(
    # this took manual tuning
    class_weight={0:0.18, 1:0.82},
    y=y_train_2
)

In [ ]:
model_2_cw = XGBClassifier(n_estimators= 150, max_depth= 4, learning_rate= 0.2, subsample= 1.0, min_child_weight= 5, gamma= 5, colsample_bytree= 1.0)

model_2_cw.fit(X_train_2, y_train_2, sample_weight=classes_weights)

In [ ]:
y_pred_cw = model_2_cw.predict(X_valid_2)


In [ ]:
from collections import Counter
# count examples in each class
counter = Counter(y_pred_cw)
print(counter[0] , counter[1])

In [ ]:
y_valid_2.value_counts()

### Evaluation - XGBoost post Class Weights (5.2)

In [ ]:
y_pred_proba_cw = model_2_cw.predict_proba(X_valid_2)[:,1]
roc_auc =  roc_auc_score(y_valid_2,y_pred_proba_cw)
y_pred_cw = model_2_cw.predict(X_valid_2)
accuracy =  accuracy_score(y_valid_2, y_pred_cw)
report = classification_report(y_valid_2, y_pred_cw, output_dict=True)

print(roc_auc, accuracy)
report



# Feature Selection (5.3)

In [ ]:
y_valid_1.nunique()

In [ ]:
## To also visualize the importance of class weight and stratifying the data, 
# the difference between the feature importance graphs could be helpful(#BLOG)

explainer_2 = shap.Explainer(model_2_cw)
shap_values_2 = explainer_2(X_train_2)

shap.summary_plot(shap_values_2, X_train_2)

### Applying Recursive Feature Elimination

In [ ]:
#adapted from https://machinelearningmastery.com/feature-importance-and-feature-selection-with-xgboost-in-python/
# from numpy import sort
from sklearn.feature_selection import SelectFromModel
#
thresholds = np.sort(model_2_cw.feature_importances_)
for thresh in thresholds:
    # select features using threshold
    selection = SelectFromModel(model_2_cw, threshold=thresh, prefit=True)
    select_X_train_2 = selection.transform(X_train_2)
    # train model
    selection_model = XGBClassifier(n_estimators= 150, max_depth= 4, learning_rate= 0.2, subsample= 1.0, min_child_weight= 5, gamma= 5, colsample_bytree= 1.0)
    selection_model.fit(select_X_train_2, y_train_2, sample_weight=classes_weights)
    # eval model
    select_X_valid_2 = selection.transform(X_valid_2)
    y_pred = selection_model.predict(select_X_valid_2)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_valid_2, predictions)
    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train_2.shape[1], accuracy*100.0))

#### Thresh = 0.320 with 1 feature yields best accuracy, but this can result being highly biased towards the majority class, so we stick with n=7

In [ ]:
model_2_cw.feature_importances_

In [ ]:
selection = SelectFromModel(model_2_cw, threshold=thresh, prefit=True)
select_X_train_2 = selection.transform(X_train_2)
    # train model
selection_model = XGBClassifier()
selection_model.fit(select_X_train_2, y_train_2, sample_weight=classes_weights)
    # eval model

# Experiment Tracking

In [ ]:
# import comet_ml
# import os

# # Check to see if there is a key in environment:
# EXPERIMENT_KEY = os.environ.get("COMET_API_KEY", None)

# # First, let's see if we continue or start fresh:
# CONTINUE_RUN = False
# if (EXPERIMENT_KEY is not None):
#     # There is one, but the experiment might not exist yet:
#     api = comet_ml.API() # Assumes API key is set in config/env
#     try:
#         api_experiment = api.get_experiment_by_id(EXPERIMENT_KEY)
#     except Exception:
#         api_experiment = None
#     if api_experiment is not None:
#         CONTINUE_RUN = True
#         # We can get the last details logged here, if logged:
#         step = int(api_experiment.get_parameters_summary("steps")["valueCurrent"])
#         epoch = int(api_experiment.get_parameters_summary("epochs")["valueCurrent"])

# if CONTINUE_RUN:
#     # 1. Recreate the state of ML system before creating experiment
#     # otherwise it could try to log params, graph, etc. again
#     # ...
#     # 2. Setup the existing experiment to carry on:
#     experiment = comet_ml.ExistingExperiment(
#         previous_experiment=EXPERIMENT_KEY,
#         log_env_details=True, # to continue env logging
#         log_env_gpu=True,     # to continue GPU logging
#         log_env_cpu=True,     # to continue CPU logging
#     )
#     # Retrieved from above APIExperiment
#     experiment.set_step(step)
#     experiment.set_epoch(epoch)

# else:
#     # 1. Create the experiment first
#     #    This will use the COMET_EXPERIMENT_KEY if defined in env.
#     #    Otherwise, you could manually set it here. If you don't
#     #    set COMET_EXPERIMENT_KEY, the experiment will get a
#     #    random key!
#     experiment = comet_ml.Experiment()
#     # 2. Setup the state of the ML system

In [ ]:
Experiment.end()